### Импорт библиотек

In [ ]:
import numpy as np
import pandas as pd
from scipy.spatial import KDTree, distance_matrix
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

### Загрузка данных

In [26]:
df = pd.read_csv('clinic-data.csv')
df.head()

,x,y,client_age,clinick_distance,density_area,park_distance,vulnerable_group_density,social_infrastructure_rating
0,52.768865,30.466943,12,1,28,52,3,8
1,55.598649,14.103713,37,1,24,16,8,7
2,61.461549,27.727829,31,3,13,59,3,8
3,51.210678,9.913478,56,1,16,29,6,4
4,62.070864,15.297742,90,3,32,90,9,5


In [27]:
client_positions = df[['x', 'y']].values
client_features = df.drop(['x', 'y'], axis=1)

In [28]:
N_CLINICS = 250
CAPACITY = 1000
ALPHA = 0.05
BETA = 0.02
GAMMA = 0.01
M = len(df)

### Расчёт метрик 

In [29]:
def calculate_metrics_exact(clinic_positions, client_positions, return_detailed=False):
    tree = KDTree(clinic_positions)
    distances, closest_indices = tree.query(client_positions)

    TTT = np.sum(ALPHA * distances + BETA * distances**2)

    clinic_loads = np.bincount(closest_indices, minlength=len(clinic_positions))

    overloads = np.maximum(0, clinic_loads - CAPACITY)
    CO = np.sum(overloads)
    
    penalty = GAMMA * CO
    T = (TTT / M) + penalty

    LCS_scores = []
    for j in range(len(clinic_positions)):
        client_idx = np.where(closest_indices == j)[0]
        if len(client_idx) > 0:
            subset = df.iloc[client_idx]
            lcs = (
                0.3 * subset['density_area'].mean() / 50 +
                0.2 * subset['social_infrastructure_rating'].mean() / 9 +
                0.2 * (1 -  subset['park_distance'].mean() / 100) +
                0.3 * subset['vulnerable_group_density'].mean() / 10
            )
            LCS_scores.append(lcs)
        else:
            LCS_scores.append(0)
    
    avg_LCS = np.mean(LCS_scores) if len(LCS_scores) > 0 else 0

    if return_detailed:
        return {
            'T': T,
            'TTT': TTT,
            'CO': CO,
            'penalty': penalty,
            'avg_distance': np.mean(distances),
            'max_distance': np.max(distances),
            'clinic_loads': clinic_loads,
            'max_load': clinic_loads.max(),
            'overloaded_clinics': np.sum(clinic_loads > CAPACITY),
            'LCS': avg_LCS,
            'LCS_scores': LCS_scores,
            'distances': distances,
            'assignments': closest_indices
        }
    else:
        return T, TTT, CO, avg_LCS

In [30]:
avg_load = M / N_CLINICS
print(f"Средняя нагрузка на клинику: {avg_load:.1f} клиентов")
print(f"Вместимость клиники: {CAPACITY}")
print(f"Коэффициент использования: {avg_load/CAPACITY*100:.1f}%")

Средняя нагрузка на клинику: 168.0 клиентов
Вместимость клиники: 1000
Коэффициент использования: 16.8%


### Weighted K-Means со штрафом за перегрузку

In [31]:
def weighted_kmeans_with_penalty(n_clusters, max_iter=100, n_init=3):
    best_centers = None
    best_score = float('inf')
    
    for init in range(n_init):
        weights = (
            df['client_age'].values / 100 +
            df['density_area'].values / 50 +
            df['vulnerable_group_density'].values / 10
        )
        weights = weights / weights.sum()
        
        indices = np.random.choice(len(df), size=min(n_clusters, 200), 
                                  p=weights, replace=False)
        init_centers = client_positions[indices]

        if len(init_centers) < n_clusters:
            additional = n_clusters - len(init_centers)
            extra = np.random.uniform(
                low=[df['x'].min(), df['y'].min()],
                high=[df['x'].max(), df['y'].max()],
                size=(additional, 2)
            )
            init_centers = np.vstack([init_centers, extra])

        for iteration in range(max_iter):
            distances = distance_matrix(client_positions, init_centers)
            assignments = np.argmin(distances, axis=1)
            
            new_centers = []
            for j in range(n_clusters):
                cluster_points = client_positions[assignments == j]
                if len(cluster_points) > 0:
                    cluster_size = len(cluster_points)
                    if cluster_size > CAPACITY:
                        center = np.mean(cluster_points, axis=0)
                        noise = np.random.normal(0, 5, size=2)
                        new_center = center + noise
                    else:
                        new_center = np.mean(cluster_points, axis=0)
                else:
                    new_center = np.random.uniform(
                        low=[df['x'].min(), df['y'].min()],
                        high=[df['x'].max(), df['y'].max()],
                        size=2
                    )
                new_centers.append(new_center)
            
            new_centers = np.array(new_centers)

            if np.allclose(init_centers, new_centers, rtol=1e-4):
                break
            
            init_centers = new_centers
        
        T, TTT, CO, LCS = calculate_metrics_exact(init_centers, client_positions)
        score = T
        
        if score < best_score:
            best_score = score
            best_centers = init_centers.copy()
    
    return best_centers

### Итеративное улучшение с помощью градиентного спуска

In [32]:
def gradient_improvement(clinic_positions, learning_rate=0.1, iterations=200):
    current = clinic_positions.copy()
    
    for iter in tqdm(range(iterations), desc="Градиентное улучшение"):
        # Текущие метрики
        tree = KDTree(current)
        distances, assignments = tree.query(client_positions)
        
        # Вычисляем "градиент" для каждой клиники
        gradient = np.zeros_like(current)
        
        for j in range(len(current)):
            # Клиенты этой клиники
            client_idx = np.where(assignments == j)[0]
            
            if len(client_idx) > 0:
                client_points = client_positions[client_idx]
                client_dists = distances[client_idx]
                
                # Градиент целевой функции: d(T)/d(position)
                # TTT часть: производная α·d + β·d²
                # Для каждого клиента: α + 2β·d в направлении к клиенту
                direction_to_clients = client_points - current[j]
                norms = np.linalg.norm(direction_to_clients, axis=1, keepdims=True)
                norms[norms == 0] = 1  # избегаем деления на 0

                 # Градиент от TTT
                ttt_grad = np.sum(
                    (ALPHA + 2 * BETA * client_dists.reshape(-1, 1)) * 
                    (direction_to_clients / norms),
                    axis=0
                ) / len(client_idx)
                
                # Градиент от штрафа за перегрузку
                overload_penalty = 0
                if len(client_idx) > CAPACITY:
                    # Если перегрузка, "отталкиваемся" от лишних клиентов
                    # Берем самых дальних клиентов
                    sorted_idx = np.argsort(-client_dists)[:len(client_idx) - CAPACITY]
                    if len(sorted_idx) > 0:
                        far_clients = client_points[sorted_idx]
                        direction_from_far = current[j] - far_clients
                        norms_far = np.linalg.norm(direction_from_far, axis=1, keepdims=True)
                        norms_far[norms_far == 0] = 1
                        overload_penalty = np.sum(direction_from_far / norms_far, axis=0)
                        overload_penalty *= GAMMA / len(sorted_idx)
                
                gradient[j] = ttt_grad + overload_penalty
        
        # Обновление позиций
        current -= learning_rate * gradient * (1 - iter/iterations)  # уменьшаем LR
        
        # Ограничение границ
        current[:, 0] = np.clip(current[:, 0], -1000, 1000)
        current[:, 1] = np.clip(current[:, 1], -1000, 1000)
    
    return current

### Алгоритм отжига

In [33]:
def simulated_annealing(initial_solution, iterations=1000):
    current = initial_solution.copy()
    current_T, current_TTT, current_CO, current_LCS = calculate_metrics_exact(
        current, client_positions
    )
    best = current.copy()
    best_T = current_T
    
    # Параметры отжига
    temperature = 10.0
    cooling_rate = 0.995
    
    history = []
    
    for i in tqdm(range(iterations), desc="Simulated Annealing"):
        neighbor = current.copy()
        
        if np.random.random() < 0.7:
            n_moves = max(1, int(N_CLINICS * 0.05))
            indices = np.random.choice(N_CLINICS, n_moves, replace=False)
            for idx in indices:
                shift = np.random.normal(0, temperature/10, size=2)
                neighbor[idx] += shift
        else:
            idx1, idx2 = np.random.choice(N_CLINICS, 2, replace=False)
            neighbor[[idx1, idx2]] = neighbor[[idx2, idx1]]

        neighbor[:, 0] = np.clip(neighbor[:, 0], -1000, 1000)
        neighbor[:, 1] = np.clip(neighbor[:, 1], -1000, 1000)
        
        neighbor_T, neighbor_TTT, neighbor_CO, neighbor_LCS = calculate_metrics_exact(
            neighbor, client_positions
        )
        
        delta_T = neighbor_T - current_T
        if delta_T < 0 or np.random.random() < np.exp(-delta_T / temperature):
            current = neighbor.copy()
            current_T = neighbor_T
            
            if neighbor_T < best_T:
                best = neighbor.copy()
                best_T = neighbor_T
        
        temperature *= cooling_rate
        history.append(best_T)
        
        if i % 200 == 0:
            print(f"Iter {i}: T={current_T:.6f}, temp={temperature:.3f}")
    
    return best, best_T, history

### Многозапусковая оптимизация

In [34]:
def multi_start_optimization(n_starts=5):
    best_solution = None
    best_metrics = None
    best_T = float('inf')
    
    for run in range(n_starts):
        print(f"\n{'='*60}")
        print(f"ЗАПУСК {run+1}/{n_starts}")
        print(f"{'='*60}")
        
        print("1. Weighted K-means инициализация...")
        init_centers = weighted_kmeans_with_penalty(N_CLINICS, max_iter=50)
        
        print("2. Градиентное улучшение...")
        gradient_improved = gradient_improvement(init_centers, iterations=100)
        
        print("3. Simulated Annealing...")
        final_solution, final_T, history = simulated_annealing(
            gradient_improved, iterations=500
        )
        
        final_metrics = calculate_metrics_exact(final_solution, client_positions, return_detailed=True)
        
        print(f"  Результат: T={final_T:.6f}, CO={final_metrics['CO']}, "
              f"LCS={final_metrics['LCS']:.6f}")

        if final_T < best_T:
            best_T = final_T
            best_solution = final_solution.copy()
            best_metrics = final_metrics.copy()
    
    return best_solution, best_metrics

### Пайплайн

In [35]:
best_solution, best_metrics = multi_start_optimization(n_starts=3)


ЗАПУСК 1/3
1. Weighted K-means инициализация...
2. Градиентное улучшение...


Градиентное улучшение: 100%|██████████| 100/100 [00:03<00:00, 25.73it/s]


3. Simulated Annealing...


Simulated Annealing:   0%|          | 2/500 [00:00<00:59,  8.35it/s]

Iter 0: T=0.058674, temp=9.950


...

Iter 200: T=0.096940, temp=3.651


Simulated Annealing:  80%|████████  | 402/500 [00:51<00:11,  8.18it/s]

Iter 400: T=0.097372, temp=1.340


Simulated Annealing: 100%|██████████| 500/500 [01:03<00:00,  7.82it/s]


  Результат: T=0.057423, CO=0, LCS=0.511443


In [36]:
print(f"Целевая функция T: {best_metrics['T']:.6f}")
print(f"Время пути TTT: {best_metrics['TTT']:.2f}")
print(f"Перегрузка CO: {best_metrics['CO']}")
print(f"Штраф: {best_metrics['penalty']:.6f}")
print(f"Коэффициент удобства LCS: {best_metrics['LCS']:.6f}")

Целевая функция T: 0.056970
Время пути TTT: 2393.10
Перегрузка CO: 0
Штраф: 0.000000
Коэффициент удобства LCS: 0.511449
